In [2]:
import pysal as ps
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
from numpy import linalg as la
%matplotlib inline
import scipy.spatial.distance as dist

In [3]:
#Pysal weights use different algorithm to compute weights that standard guassian weights from GWR
#Below is a simple fixed gaussain bandwidth based on code from GWR4 which matche GWR4 and spgwr output

bandwidth = 3
data = ps.open(ps.examples.get_path('columbus.dbf'))
coords = zip(data.by_col['X'], data.by_col['Y'])
dists = dist.squareform(dist.pdist(coords))
W = np.exp(-0.5*(dists/bandwidth)**2)

In [7]:
W[0].shape

(49,)

In [236]:
y = data.by_col_array(['HOVAL'])
X1, X2 = data.by_col_array(['INC']), data.by_col_array(['CRIME'])

In [237]:
X = np.hstack((X1, X2))

In [238]:
colshp = ps.open(ps.examples.get_path('columbus.shp')).read()
colshp_coords = np.array([x.centroid for x in colshp])

In [239]:
design = np.hstack((y, X))

In [240]:
data2 = pd.DataFrame(design, columns=['y', 'X1', 'X2'])

In [241]:
X2.shape, X1.shape, y.shape

((49, 1), (49, 1), (49, 1))

## Python Estimation

In [242]:
X = np.hstack((np.ones(X1.shape), X))

In [243]:
class BaseGWR(object):
    def __init__(self, y, X, W):
        self.y = y
        self.n = y.shape[0]
        self.p = X.shape[-1]
        self.X = X
        self.W = W
        self.vm = NotImplementedError
        self.betas = []
        self.yhats = []
        
        for i in range(self.n):
            Wuv = np.diag(self.W[i])
            xtw = np.dot(self.X.T, Wuv)
            xtwx = np.dot(xtw, self.X)
            xtwx_inv = la.inv(xtwx)
            xtwx_inv_xtw = np.dot(xtwx_inv, xtw)
            beta_hats = np.dot(xtwx_inv_xtw, self.y)
            self.betas.append(beta_hats.T)
            yhat = np.dot(self.X[i,:], beta_hats)
            self.yhats.append(yhat[0])
        self.betas = np.array(self.betas).flatten()
        self.yhats = np.array(self.yhats).reshape(self.y.shape)
            
        self.SSE = np.sum((self.y - self.yhats)**2)
        self.MSE = self.SSE / float(self.n)
        self.SST = np.sum((self.y - np.mean(self.y))**2)
        self.SSR = self.SST - self.SSE
        self.R2 = self.SSR/self.SST
        
        self.MSR = self.SSR / self.p
        self.MST = self.SST / (self.n - self.p - 1)
        self.F = self.MSR/self.MST
        
    def __repr__(self):
        return "This is a GWR model"

In [244]:
mod = BaseGWR(y, X, W)

In [246]:
mod.betas[0:3]

array([ 86.53224885,   0.16213264,  -1.53927348])

In [247]:
mod

This is a GWR model

In [226]:
import scipy.stats as st

In [227]:
1 - st.f.cdf(mod.F, mod.n-1, mod.n-mod.p-1)

2.1027624086400465e-13

In [228]:
classical = ps.spreg.OLS(y, np.hstack((X1,X2)))

In [229]:
classical.f_stat

(12.35819888535632, 5.0636903313961772e-05)

In [230]:
mod.F

10.739817104530468